# Buffer circulaire

Si l'insertion en queue d'un tableau de capacité fixe est efficace, c'est parce que la position de la fin du tableau est variable. Elle est contrôlée par la variable `taille`.

Par contre, le début est fixe en position `0`, ce qui rend l'insertion en tête coûteuse. 

## Principe 

On étend la notion de tableau de capacité fixe en rendant la position du début variable.

Quatre attributs sont alors nécessaires à cette structure

* l'addresse _constante_ de début de mémoire allouée
* la capacité _constante_
* la taille _variable_
* la position de début _variable_

In [1]:
import include.buffer_circulaire as h

In [2]:
class BufferCirculaire:
    def __init__(self,capacite):
        self.data = [None]*capacite
        self.capacite = capacite
        self.taille = 0
        self.debut = 0
    
    __str__     = h.convertir_en_texte

## Indices 

Chaque emplacement en mémoire a 2 indices distincts
* un indice **physique** qui donne sa position em mémoire
* un indice **logique** qui donne sa position depuis `self.debut`.
![buffer circulaire](include/circulaire_1.png)

In [3]:
def indice_physique(B,i_logique): return B.debut + i_logique

def indice_logique(B,i_physique): return i_physique - B.debut 

Mieux, on utilise la mémoire allouée de manière cyclique, en calculant les indices physiques modulo la capacité.

![buffer circulaire](include/circulaire_2.png)

In [4]:
def indice_physique(B,i_logique):
    return ( B.debut + i_logique ) % B.capacite

def indice_logique(B,i_physique):
    return ( B.capacite + i_physique - B.debut ) % B.capacite

Cela permet d'utiliser toute la mémoire allouée. 

Si nécessaire, i.e. quand `debut + taille >= capacite`, 

Les indices logiques `[0,taille[` correspondent

aux indices physiques `[debut,capacite[`, 

suivis de `[0,debut + taille - capacite[`

In [5]:
B = h.BufferDemo()
print("capacité:         ",B.capacite)
print("taille:           ",B.taille)
print("debut:            ",B.debut)
print("tableau physique: ",B.data)
print("tableau logique:  ", end=" ")

for i in range(B.taille):
    print(B.data[indice_physique(B,i)],end=" ")

capacité:          6
taille:            5
debut:             4
tableau physique:  [4, 9, 16, None, 0, 1]
tableau logique:   0 1 4 9 16 

In [6]:
print(B)

0 1 4 9 16  | d: 4 | t/c: (5/6) | [4, 9, 16, None, 0, 1]


## Opérations

Ces fonctions de traduction entre indices logiques et physiques nous permettent d'écrire aisément les opérations essentielles du buffer circulaire, toutes de complexité $\Theta(1)$

* Insertion en tête et en queue


* Suppression en tête et en queue


* Accès à la tête, la queue, un élément quelconque

### Insertions

Pour insérer en queue, il suffit d'écrire à l'emplacement d'indice logique `B.taille` et d'incrémenter la taille

In [7]:
def inserer_en_queue(B,valeur):
    if B.taille >= B.capacite: raise IndexError("")
        
    B.data[indice_physique(B,B.taille)] = valeur
    B.taille += 1

Pour insérer en tête, il faut écrire à l'indice logique `-1` et déplacer le `debut` en plus d'incrémenter la taille

In [8]:
def inserer_en_tete(B,valeur):
    if B.taille >= B.capacite: raise IndexError("")
        
    B.debut = indice_physique(B,-1)
    B.data[B.debut] = valeur
    B.taille += 1

In [9]:
B = BufferCirculaire(4)
print(B)
for i in range(4):
    if i%2:
        inserer_en_queue(B,i)
    else:
        inserer_en_tete(B,i)
    print(B)

Rien       | d: 0 | t/c: (0/4) | [None, None, None, None]
0          | d: 3 | t/c: (1/4) | [None, None, None, 0]
0 1        | d: 3 | t/c: (2/4) | [1, None, None, 0]
2 0 1      | d: 2 | t/c: (3/4) | [1, None, 2, 0]
2 0 1 3    | d: 2 | t/c: (4/4) | [1, 3, 2, 0]


### Suppressions

Les suppressions fonctionnent de manière similaire. Elles 

* détruisent l'élément en tête ou en queue
* décrémentent la taille

et la suppression en tête incrémente l'indice physique du `debut`.

In [10]:
def supprimer_en_queue(B):
    if B.taille <= 0: raise IndexError("")
        
    B.data[indice_physique(B,B.taille-1)] = None
    B.taille -= 1

In [11]:
def supprimer_en_tete(B):
    if B.taille <= 0: raise IndexError("")
        
    B.data[B.debut] = None
    B.debut = indice_physique(B,1)
    B.taille -= 1

In [12]:
B = BufferCirculaire(3)
print(B)
for i in range(4):
    inserer_en_queue(B,i)
    print(B)
    if i%2:
        supprimer_en_tete(B)
        print(B)

Rien       | d: 0 | t/c: (0/3) | [None, None, None]
0          | d: 0 | t/c: (1/3) | [0, None, None]
0 1        | d: 0 | t/c: (2/3) | [0, 1, None]
1          | d: 1 | t/c: (1/3) | [None, 1, None]
1 2        | d: 1 | t/c: (2/3) | [None, 1, 2]
1 2 3      | d: 1 | t/c: (3/3) | [3, 1, 2]
2 3        | d: 2 | t/c: (2/3) | [3, None, 2]


In [13]:
B = BufferCirculaire(3)
print(B)
for i in range(4):
    inserer_en_tete(B,i)
    print(B)
    if i%2:
        supprimer_en_queue(B)
        print(B)

Rien       | d: 0 | t/c: (0/3) | [None, None, None]
0          | d: 2 | t/c: (1/3) | [None, None, 0]
1 0        | d: 1 | t/c: (2/3) | [None, 1, 0]
1          | d: 1 | t/c: (1/3) | [None, 1, None]
2 1        | d: 0 | t/c: (2/3) | [2, 1, None]
3 2 1      | d: 2 | t/c: (3/3) | [2, 1, 3]
3 2        | d: 2 | t/c: (2/3) | [2, None, 3]


### Accès aux éléments

In [14]:
def getitem(B,i):
    if i >= B.taille or i < 0: 
        raise IndexError("")
    return B.data[indice_physique(B,i)]

In [15]:
def setitem(B,i,valeur):
    if i >= B.taille or i < 0: 
        raise IndexError("")
    B.data[indice_physique(B,i)] = valeur 

In [16]:
def tete(B):   
    return getitem(B,0)

In [17]:
def queue(B):
     return getitem(B,B.taille-1)

## Classe python

In [18]:
class BufferCirculaire:
    def __init__(self,capacite):
        self.data = [None]*capacite; self.capacite = capacite
        self.taille = 0; self.debut = 0
    
    __str__     = h.convertir_en_texte
    __getitem__ = getitem
    __setitem__ = setitem
    append      = inserer_en_queue
    appendleft  = inserer_en_tete
    
    def __len__(self): return self.taille
    def maxlen(self):  return self.capacite
    def pop(self): 
        tmp = queue(self)
        supprimer_en_queue(self)
        return tmp; 
    def popleft(self):
        tmp = tete(self)
        supprimer_en_tete(self)
        return tmp;

Cette classe peut-être utilisée comme une queue FIFO ou comme une queue à deux fins.

In [19]:
B = BufferCirculaire(6)
for i in range(5):
    B.append(i)

for i in range(10):
    print(B.popleft(), end = " ")
    B.append(i+10)

print()
for i in B:
    print(i, end = " ")

0 1 2 3 4 10 11 12 13 14 
15 16 17 18 19 

<table style="width: 100%; border: 0px">
<tr style="background-color:white; border:0px">
<td style="width: 120px; border: 0px">
    <img src="https://heig-vd.ch/ResourcePackages/WhiteFox/assets/images/logo-heig-vd.svg" height=200px align=left >
    </td>
    <td style="vertical-align: middle; border: 0px" height=200px>
    <p style="text-align: left">
        <a href="https://ocuisenaire.github.io/ASD1-notebooks/">ASD1 Notebooks on GitHub.io</a>
 </p>        
<p style="text-align: left">
© Olivier Cuisenaire, 2018 </p>
</td>
</tr>
</table>